In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

torch.cuda.empty_cache()

Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import torch
import numpy as np
import os
import torch.nn as nn
from tqdm import tqdm
import json
from functools import partial
from torch import einsum, nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from einops import pack, rearrange, reduce, repeat, unpack


In [4]:
def findAllFile(base):
    file_path = []
    for root, ds, fs in os.walk(base, followlinks=True):
        for f in fs:
            fullname = os.path.join(root, f)
            file_path.append(fullname)
    return file_path



In [5]:
import scipy.ndimage.filters as filters
from utils.motion_processing.quaternion import (
    cont6d_to_matrix,
    cont6d_to_matrix_np,
    qbetween_np,
    qinv_np,
    qmul,
    qmul_np,
    qrot,
    qrot_np,
)


In [6]:

smplx_raw_body_offsets = np.array(
    [
        [0, 0, 0],  # pelvis
        [1, 0, 0],  # left_hip
        [-1, 0, 0],  # right_hip
        [0, 1, 0],  # spine1
        [0, -1, 0],  # left_knee
        [0, -1, 0],  # right_knee
        [0, 1, 0],  # spine2
        [0, -1, 0],  # left_ankle
        [0, -1, 0],  # right_ankle
        [0, 1, 0],  # spine3
        [0, 0, 1],  # left_foot
        [0, 0, 1],  # right_foot
        [0, 1, 0],  # neck
        [1, 0, 0],  # left_collar
        [-1, 0, 0],  # right_collar
        [0, 0, 1],  # head
        [0, -1, 0],  # left_shoulder
        [0, -1, 0],  # right_shoulder
        [0, -1, 0],  # left_elbow
        [0, -1, 0],  # right_elbow
        [0, -1, 0],  # left_wrist
        [0, -1, 0],  # right_wrist
    ]
)

l_idx1, l_idx2 = 5, 8
# Right/Left foot
fid_r, fid_l = [8, 11], [7, 10]
# Face direction, r_hip, l_hip, sdr_r, sdr_l
face_joint_indx = [2, 1, 17, 16]
r_hip, l_hip = 2, 1

In [7]:
from core.datasets.vq_dataset import VQSMPLXMotionDataset
from core.datasets.vq_dataset import load_dataset, simple_collate


/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
dset = VQSMPLXMotionDataset("choreomaster" , "/srv/hays-lab/scratch/sanisetty3/motionx" ,hml_rep = "gpvrc" , window_size_s=4, motion_rep = "body", split = "train" , fps = 30  )


Total number of motions choreomaster: 34


In [46]:
from core.datasets.kinematics import smplx_full_kinematic_chain

In [49]:
(smplx_full_kinematic_chain)

[[0, 1, 4, 7, 10],
 [0, 2, 5, 8, 11],
 [0, 3, 6, 9, 12, 15],
 [9, 13, 16, 18, 20],
 [9, 14, 17, 19, 21],
 [20, 22, 23, 24],
 [20, 25, 26, 27],
 [20, 28, 29, 30],
 [20, 31, 32, 33],
 [20, 34, 35, 36],
 [21, 37, 38, 39],
 [21, 40, 41, 42],
 [21, 43, 44, 45],
 [21, 46, 47, 48],
 [21, 49, 50, 51]]

In [85]:
re_order = [0]
for i in smplx_full_kinematic_chain:
    re_order.extend(i[1:])

In [86]:
dict(zip(re_order , template.parents))

{0: -1,
 1: 0,
 4: 1,
 7: 2,
 10: 3,
 2: 0,
 5: 5,
 8: 6,
 11: 7,
 3: 0,
 6: 9,
 9: 10,
 12: 11,
 15: 12,
 13: 11,
 16: 14,
 18: 15,
 20: 16,
 14: 17,
 17: 18,
 19: 19,
 21: 17,
 22: 21,
 23: 22,
 24: 17,
 25: 24,
 26: 25,
 27: 17,
 28: 27,
 29: 28,
 30: 17,
 31: 30,
 32: 31,
 33: 11,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 36,
 41: 40,
 42: 41,
 43: 36,
 44: 43,
 45: 44,
 46: 36,
 47: 46,
 48: 47,
 49: 36,
 50: 49,
 51: 50}

In [75]:
len(re_order)

52

In [76]:
len(template.parents)

52

In [78]:
template.parents

array([-1,  0,  1,  2,  3,  0,  5,  6,  7,  0,  9, 10, 11, 12, 11, 14, 15,
       16, 17, 18, 19, 17, 21, 22, 17, 24, 25, 17, 27, 28, 17, 30, 31, 11,
       33, 34, 35, 36, 37, 38, 36, 40, 41, 36, 43, 44, 36, 46, 47, 36, 49,
       50])

{0: -1,
 1: 0,
 4: 1,
 7: 2,
 10: 3,
 2: 0,
 5: 5,
 8: 6,
 11: 7,
 3: 0,
 6: 9,
 9: 10,
 12: 11,
 15: 12,
 13: 11,
 16: 14,
 18: 15,
 20: 16,
 14: 17,
 17: 18,
 19: 19,
 21: 17,
 22: 21,
 23: 22,
 24: 17,
 25: 24,
 26: 25,
 27: 17,
 28: 27,
 29: 28,
 30: 17,
 31: 30,
 32: 31,
 33: 11,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 36,
 41: 40,
 42: 41,
 43: 36,
 44: 43,
 45: 44,
 46: 36,
 47: 46,
 48: 47,
 49: 36,
 50: 49,
 51: 50}

In [ ]:
0,1,4,7,19,2, 5, 8, 11,3, 6, 9, 12, 15,13, 16, 18, 20,22, 23, 24, 25, 26, 27, 28, 29, 30, 31,32,33,34,35,36,14, 17, 19, 21,37, 38, 39,40,41,42,43,44,45,46,47,48,49,50,51

In [ ]:
# pelvis ---> left_hip ---> left_knee ---> left_ankle ---> left_foot 0, 1, 4, 7, 10
# pelvis --> right_hip ---> right_knee ---> right_ankle ---> right_foot 0, 2, 5, 8, 11
# pelvis ---> spine1 ---> spine2 ---> spine3 ---> neck ---> head 0, 3, 6, 9, 12, 15
# spine3 ---> left_collar ---> left_shoulder --->  left_elbow ---> left_wrist 9, 13, 16, 18, 20
# spine3 ---> right_collar ---> right_shoulder --->  right_elbow ---> right_wrist  9, 14, 17, 19, 21

smplx_kinematic_chain = [
    [0, 1, 4, 7, 10],
    [0, 2, 5, 8, 11],
    [0, 3, 6, 9, 12, 15],
    [9, 13, 16, 18, 20],
    [9, 14, 17, 19, 21],
]

# left_wrist ---> left_index1 ---> left_index2 --->left_index3
# left_wrist ---> left_middle1 ---> left_middle2 --->left_middle3
# left_wrist ---> left_pinky1 ---> left_pinky2 --->left_pinky3
# left_wrist ---> left_ring1 ---> left_ring2 --->left_ring3
# left_wrist ---> left_thumb1 ---> left_thumb2 --->left_thumb3


smplx_left_hand_chain = [
    [20, 22, 23, 24],
    [20, 25, 26, 27],
    [20, 28, 29, 30],
    [20, 31, 32, 33],
    [20, 34, 35, 36],
]

# right_wrist ---> right_index1 ---> right_index2 --->right_index3
# right_wrist ---> right_middle1 ---> right_middle2 --->right_middle3
# right_wrist ---> right_pinky1 ---> right_pinky2 --->right_pinky3
# right_wrist ---> right_ring1 ---> right_ring2 --->right_ring3
# right_wrist ---> right_thumb1 ---> right_thumb2 --->right_thumb3
smplx_right_hand_chain = [
    [21, 37, 38, 39],
    [21, 40, 41, 42],
    [21, 43, 44, 45],
    [21, 46, 47, 48],
    [21, 49, 50, 51],
]


In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import render_bvh.BVH_mod as BVH
from render_bvh.joints2bvh2 import Joint2BVHConvertor

In [150]:
from render_bvh.joints2bvh2 import Joint2BVHConvertor
converter = Joint2BVHConvertor()
template = BVH.load("./render_bvh/data/smplx_eulerxyz2.bvh", need_quater=True)


In [106]:
re_order = [0]
for i in smplx_full_kinematic_chain:
    re_order.extend(i[1:])

In [124]:
print(re_order)

[0, 1, 4, 7, 10, 2, 5, 8, 11, 3, 6, 9, 12, 15, 13, 16, 18, 20, 14, 17, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


In [123]:
template.parents

array([-1,  0,  1,  2,  3,  0,  5,  6,  7,  0,  9, 10, 11, 12, 11, 14, 15,
       16, 17, 18, 19, 17, 21, 22, 17, 24, 25, 17, 27, 28, 17, 30, 31, 11,
       33, 34, 35, 36, 37, 38, 36, 40, 41, 36, 43, 44, 36, 46, 47, 36, 49,
       50])

In [125]:
re_order_inv = list(
            dict(sorted(dict(zip(re_order, np.arange(0, 52))).items())).values()
        )
print(re_order_inv)

[0, 1, 5, 9, 2, 6, 10, 3, 7, 11, 4, 8, 12, 14, 18, 13, 15, 19, 16, 20, 17, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


In [149]:
template_og = BVH.load("./render_bvh/data/template.bvh", need_quater=True)

In [128]:
list(template_og.parents)

[-1, 0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 12, 11, 14, 15, 16, 11, 18, 19, 20]

In [138]:
smpl_chain = [0, 1, 4, 7, 10, 0, 2, 5, 8, 11, 0, 3, 6, 9, 12, 15, 9, 13, 16, 18, 20, 9, 14, 17, 19, 21] ##chain

In [139]:
reorder = [0, 1, 4, 7, 10, 2, 5, 8, 11, 3, 6, 9, 12, 15, 13, 16, 18, 20, 14, 17, 19, 21] ##reorder

In [147]:
print(template.names)

['pelvis', 'left_hip', 'left_knee', 'left_ankle', 'left_foot', 'right_hip', 'right_knee', 'right_ankle', 'right_foot', 'spine1', 'spine2', 'spine3', 'neck', 'head', 'left_collar', 'left_shoulder', 'left_elbow', 'left_wrist', 'left_index1', 'left_index2', 'left_index3', 'left_middle1', 'left_middle2', 'left_middle3', 'left_pinky1', 'left_pinky2', 'left_pinky3', 'left_ring1', 'left_ring2', 'left_ring3', 'left_thumb1', 'left_thumb2', 'left_thumb3', 'right_collar', 'right_shoulder', 'right_elbow', 'right_wrist', 'right_index1', 'right_index2', 'right_index3', 'right_middle1', 'right_middle2', 'right_middle3', 'right_pinky1', 'right_pinky2', 'right_pinky3', 'right_ring1', 'right_ring2', 'right_ring3', 'right_thumb1', 'right_thumb2', 'right_thumb3']


In [69]:
template.parents

array([-1,  0,  1,  2,  3,  0,  5,  6,  7,  0,  9, 10, 11, 12, 11, 14, 15,
       16, 17, 18, 19, 17, 21, 22, 17, 24, 25, 17, 27, 28, 17, 30, 31, 11,
       33, 34, 35, 36, 37, 38, 36, 40, 41, 36, 43, 44, 36, 46, 47, 36, 49,
       50])

In [ ]:
# pelvis --> right_hip ---> right_knee ---> right_ankle ---> right_foot 0, 2, 5, 8, 11
# pelvis ---> left_hip ---> left_knee ---> left_ankle ---> left_foot 0, 1, 4, 7, 10
# pelvis ---> spine1 ---> spine2 ---> spine3 ---> neck ---> head 0, 3, 6, 9, 12, 15
# spine3 ---> right_collar ---> right_shoulder --->  right_elbow ---> right_wrist  9, 14, 17, 19, 21
# spine3 ---> left_collar ---> left_shoulder --->  left_elbow ---> left_wrist 9, 13, 16, 18, 20

smplx_kinematic_chain = [
    [0, 1, 4, 7, 10],
    [0, 2, 5, 8, 11],
    [0, 3, 6, 9, 12, 15],
    [9, 13, 16, 18, 20],
    [9, 14, 17, 19, 21],
]

In [6]:
from core.datasets.vq_dataset import VQSMPLXMotionDataset
from core.datasets.vq_dataset import load_dataset, simple_collate
import render_bvh.BVH_mod as BVH


/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dset = VQSMPLXMotionDataset("choreomaster" , "/srv/hays-lab/scratch/sanisetty3/motionx" ,hml_rep = "gpvc" , window_size=-1, motion_rep = "full", split = "train" , fps = 30  )


Total number of motions choreomaster: 34


In [8]:

motion  = dset[12]["motion"]
print(dset[12]["name"])
motion_inv = dset.inv_transform(motion)
motion_xyz = dset.to_xyz(motion_inv)[:300]

choreomaster/5034.npy


In [9]:
motion_xyz.shape

torch.Size([300, 52, 3])

In [65]:


from render_bvh.joints2bvh import Joint2BVHConvertor
converter = Joint2BVHConvertor()
template = BVH.load("./render_bvh/data/template.bvh", need_quater=True)


In [66]:
 _, ik_joint = converter.convert(motion_xyz.cpu().numpy(), filename="/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render_bvh/data/template_render.bvh", iterations=10, foot_ik = False)

In [23]:


from render_bvh.joints2bvh import Joint2BVHConvertor
converter = Joint2BVHConvertor("./render_bvh/data/smplx_body.bvh")
# template = BVH.load("./render_bvh/data/smplx_body.bvh", need_quater=True)


In [80]:
_, ik_joint = converter.convert(motion_xyz.cpu().numpy(), filename="/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render_bvh/data/smplx_body_render.bvh", iterations=10, foot_ik = False)

In [29]:
import utils.vis_utils.plot_3d_global as plot_3d


In [25]:
from render_bvh.joints2bvh2 import Joint2BVHConvertor
converter = Joint2BVHConvertor(path="./render_bvh/data/smplx_full_template.bvh")
template = BVH.load("./render_bvh/data/smplx_full_template.bvh", need_quater=True)

In [26]:
_, ik_joint = converter.convert(motion_xyz.cpu().numpy(), filename="/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render_bvh/data/smplx_full_render.bvh", iterations=10, foot_ik = False)

In [28]:
ik_joint.shape

(300, 52, 3)

In [30]:
plot_3d.render(
            np.array(ik_joint),
            "/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render_bvh/data/smplx_full_render.gif",
        )

In [27]:
print(template.names)

['pelvis', 'left_hip', 'left_knee', 'left_ankle', 'left_foot', 'right_hip', 'right_knee', 'right_ankle', 'right_foot', 'spine1', 'spine2', 'spine3', 'neck', 'head', 'left_collar', 'left_shoulder', 'left_elbow', 'left_wrist', 'left_index1', 'left_index2', 'left_index3', 'left_middle1', 'left_middle2', 'left_middle3', 'left_pinky1', 'left_pinky2', 'left_pinky3', 'left_ring1', 'left_ring2', 'left_ring3', 'left_thumb1', 'left_thumb2', 'left_thumb3', 'right_collar', 'right_shoulder', 'right_elbow', 'right_wrist', 'right_index1', 'right_index2', 'right_index3', 'right_middle1', 'right_middle2', 'right_middle3', 'right_pinky1', 'right_pinky2', 'right_pinky3', 'right_ring1', 'right_ring2', 'right_ring3', 'right_thumb1', 'right_thumb2', 'right_thumb3']


In [17]:
print(template.parents)

[-1  0  1  2  3  0  5  6  7  0  9 10 11 12 11 14 15 16 17 18 19 17 21 22
 17 24 25 17 27 28 17 30 31 11 33 34 35 36 37 38 36 40 41 36 43 44 36 46
 47 36 49 50]


In [22]:
print(converter.re_order)

[0, 1, 4, 7, 10, 2, 5, 8, 11, 3, 6, 9, 12, 15, 13, 16, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 14, 17, 19, 21, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


[0, 1, 5, 9, 2, 6, 10, 3, 7, 11, 4, 8, 12, 14, 18, 13, 15, 19, 16, 20, 17, 21]

In [15]:
reorder_inv = ([0, 1, 5, 9, 2, 6, 10, 3, 7, 11 , 4 , 8, 12, 14, 33, 13, 15, 34, 16, 35, 17, 36, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 37,
            38,
            39,
            40,
            41,
            42,
            43,
            44,
            45,
            46,
            47,
            48,
            49,
            50,
            51,])

In [16]:
dict(zip(np.arange(52) , reorder_inv))

{0: 0,
 1: 1,
 2: 5,
 3: 9,
 4: 2,
 5: 6,
 6: 10,
 7: 3,
 8: 7,
 9: 11,
 10: 4,
 11: 8,
 12: 12,
 13: 14,
 14: 33,
 15: 13,
 16: 15,
 17: 34,
 18: 16,
 19: 35,
 20: 17,
 21: 36,
 22: 18,
 23: 19,
 24: 20,
 25: 21,
 26: 22,
 27: 23,
 28: 24,
 29: 25,
 30: 26,
 31: 27,
 32: 28,
 33: 29,
 34: 30,
 35: 31,
 36: 32,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51}

In [ ]:
[0, 1, 5, 9, 2, 6, 10, 3, 7, 12, 14, 33, 13, 15, 34, 16, 35, 17, 36, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]7